<h4>SVM</h4>

<h5>Imports</h5>

In [1]:
from urllib.parse import urlparse, parse_qs, urlencode, urlunparse, unquote_plus
import tensorflow as tf
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, Input, Dropout
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.models import Model
from tensorflow.keras import layers, losses
import jsonlines
import os
import json
import pickle
from sklearn.metrics import f1_score, accuracy_score, precision_score
from sklearn import svm
from tensorflow.python.profiler import profiler_v2 as profiler

2023-05-21 15:43:29.576732: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-21 15:43:29.622698: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-21 15:43:30.134423: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
file_names = ['dataset/malicious_clean.txt' ,'dataset/normal_clean.txt']
data={}
for file in file_names:
    data[file]=[]
    with jsonlines.open(file) as reader:
        for line in reader:
            if line['request']['method'] == 'POST':
                data[file].append('POST'+' '+line['request']['uri']+' '+line['request']['body']+' '+json.dumps(line['request']['headers']))
            else:
                data[file].append('GET'+' '+line['request']['uri']+' '+json.dumps(line['request']['headers']))
                
                
normal = data[file_names[1]]
malicious = data[file_names[0]]
normal_part1 = normal[180000:]
normal_part2 = normal[:180000]

train_examples = normal_part2
test_examples = normal_part1+malicious
train_labels = [0] * len(train_examples)
test_labels = [0]* len(normal_part1)
test_labels.extend([1] * len(malicious))

dataset_train = tf.data.Dataset.from_tensor_slices((list(map(lambda x: unquote_plus(x),train_examples)), train_labels))
dataset_test = tf.data.Dataset.from_tensor_slices((list(map(lambda x: unquote_plus(x),test_examples)), test_labels))


def preprocess_text(text, label):
    text = tf.strings.lower(text)
    text = tf.strings.regex_replace(text, "[a-zA-Z]+", "a")
    text = tf.strings.regex_replace(text, "[0-9]+", "n")
    text = tf.strings.regex_replace(text, "(a|n){2,}", "x")
    text = tf.strings.regex_replace(text, '[^\x00-\x7F]+', '')
    text = tf.strings.regex_replace(text, '(.)',  r'\1 ')
    return text, label

def preprocess_text_substitution(text, label):
    text = tf.strings.lower(text)
    text = tf.strings.regex_replace(text, "[a-zA-Z]+", "a")
    text = tf.strings.regex_replace(text, "[0-9]+", "n")
    text = tf.strings.regex_replace(text, "(a|n){2,}", "x")
    text = tf.strings.regex_replace(text, '[^\x00-\x7F]+', '')
    punctuation = "=?/(){}[]<>"
    for p in punctuation:
        text = tf.strings.regex_replace(text, "\\" + p, " "+p+" ")
    return text, label

 
# Map the preprocess function to the dataset
train = dataset_train.map(preprocess_text)
test = dataset_test.map(preprocess_text)

train_sub = dataset_train.map(preprocess_text_substitution)
test_sub = dataset_test.map(preprocess_text_substitution)

for text, label in train_sub.take(1):
    print(text.numpy(), label.numpy())

for text, label in train.take(1):
    print(text.numpy(), label.numpy())

2023-05-21 15:43:35.547463: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-21 15:43:35.687117: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-21 15:43:35.687162: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-21 15:43:35.688285: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-21 15:43:35.688324: I tensorflow/compile

b'a  / a / a / a  { "a" = "a" }   { "a-a": "a", "a": "a / a,* / *", "a-a": "a, a, a", "a-a": "a / n.n  ( a a n.n; x; x )  a / n.n  ( a, a a )  a / n.n.n.n a / n.n", "a": "a = x; a = x" } ' 0
b'a   / a / a / a   { " a " = " a " }   { " a - a " :   " a " ,   " a " :   " a / a , * / * " ,   " a - a " :   " a ,   a ,   a " ,   " a - a " :   " a / n . n   ( a   a   n . n ;   x ;   x )   a / n . n   ( a ,   a   a )   a / n . n . n . n   a / n . n " ,   " a " :   " a = x ;   a = x " } ' 0


2023-05-21 15:43:37.630281: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int32 and shape [180000]
	 [[{{node Placeholder/_1}}]]
2023-05-21 15:43:37.648130: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int32 and shape [180000]
	 [[{{node Placeholder/_1}}]]


In [3]:
from_disk = pickle.load(open("vectorization.pkl", "rb"))
vectorization = TextVectorization.from_config(from_disk['config'])
# You have to call `adapt` with some dummy data (BUG in Keras)
vectorization.adapt(tf.data.Dataset.from_tensor_slices(["xyz"]))
vectorization.set_weights(from_disk['weights'])

from_disk_sub = pickle.load(open("vectorization_sub.pkl", "rb"))
vectorization_sub = TextVectorization.from_config(from_disk_sub['config'])
# You have to call `adapt` with some dummy data (BUG in Keras)
vectorization_sub.adapt(tf.data.Dataset.from_tensor_slices(["xyz"]))
vectorization_sub.set_weights(from_disk_sub['weights'])

2023-05-21 15:43:37.706341: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [1]
	 [[{{node Placeholder/_0}}]]
2023-05-21 15:43:37.813481: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [1]
	 [[{{node Placeholder/_0}}]]


In [4]:


def sub_processing(text, label):
    return  vectorization_sub(text), label

def processing(text, label):
    return  vectorization(text), label

train_auto_sub = train_sub.map(sub_processing)
train_auto = train.map(processing)

test_auto_sub = test_sub.map(sub_processing)
test_auto = test.map(processing)


# convert tf Dataset to numpy array
train_auto = np.array(list(train_auto.as_numpy_iterator()))
test_auto = np.array(list(test_auto.as_numpy_iterator()))
train_auto_sub = np.array(list(train_auto_sub.as_numpy_iterator()))
test_auto_sub = np.array(list(test_auto_sub.as_numpy_iterator()))

#extract the text and label from the numpy array
train_text = train_auto[:,0]
train_label = train_auto[:,1]
test_text = test_auto[:,0]
test_label = test_auto[:,1]

train_text_sub = train_auto_sub[:,0]
train_label_sub = train_auto_sub[:,1]
test_text_sub = test_auto_sub[:,0]
test_label_sub = test_auto_sub[:,1]

2023-05-21 15:43:38.080719: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_7' with dtype int64
	 [[{{node Placeholder/_7}}]]
/tmp/ipykernel_342/3878776300.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  train_auto = np.array(list(train_auto.as_numpy_iterator()))
2023-05-21 15:44:10.258043: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_7' with dtype int64
	 [[{{node Pl

In [5]:

import time
start = time.time()
model = svm.OneClassSVM(gamma='auto').fit(list(train_text)[:180000])
end = time.time()
print(end - start)
start = time.time()
model_sub = svm.OneClassSVM(gamma='auto').fit(list(train_text_sub)[:180000])
end = time.time()
print(end - start)

7288.655417919159
2652.528799057007


In [5]:
import pickle
filename = './svm/svm_model.sav'
#pickle.dump(model, open(filename, 'wb'))
model = pickle.load(open(filename, 'rb'))

filename_sub = './svm_sub/svm_model.sav'
#pickle.dump(model_sub, open(filename_sub, 'wb'))
model_sub = pickle.load(open(filename_sub, 'rb'))

In [7]:
import time
start_time = time.time()
pred = model.predict(list(test_text))
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time} seconds")
start_time = time.time()
pred_sub = model_sub.predict(list(test_text_sub))
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time} seconds")

Elapsed time: 2124.5850760936737 seconds
Elapsed time: 1110.4105966091156 seconds


In [12]:
pred_t = list(map(lambda x: 1 if x==-1 else 0, pred))
pred_sub_t = list(map(lambda x: 1 if x==-1 else 0, pred_sub))

In [32]:
len(list(pred_sub_t))

117899

In [25]:
print("Metrics")
print("f1: ", f1_score(list(test_label),pred_t))
print("acc: ", accuracy_score(list(test_label),pred_t))
print("prec: ", precision_score(list(test_label),pred_t))

Metrics
f1:  0.8864596405970468
acc:  0.8196676816597257
prec:  0.8221106224493839


In [33]:
print("Metrics")
print("f1: ", f1_score(list(test_label_sub),pred_sub_t))
print("acc: ", accuracy_score(list(test_label_sub),pred_sub_t))
print("prec: ", precision_score(list(test_label_sub),pred_sub_t))

Metrics
f1:  0.886188162802701
acc:  0.8185820066327959
prec:  0.8193321132691815
